# InSTA: Towards Internet-Scale Training For Agents

![Pipeline Overview](https://data-for-agents.github.io/static/images/pipeline_overview.png)

**Brandon Trabucco (1) Gunnar Sigurdsson (2) Robinson Piramuthu (2) Ruslan Salakhutdinov (1)**

**(1) Carnegie Mellon University, Machine Learning Department (2) Amazon**

The predominant approach for training web navigation agents gathers human demonstrations for a set of popular websites and hand-written tasks, but it is becoming clear that human data are an inefficient resource. We develop a pipeline to facilitate Internet-scale training for agents without laborious human annotations. In the first stage, an LLM generates tasks for 150k diverse websites. In the next stage, LLM agents complete tasks and produce trajectories. In the final stage, an LLM reviews the trajectories and judges their success. Language models are competitive with human annotators, detecting and filtering out harmful content with an accuracy of 97%, generating feasible tasks with an 89% rate, and judging successful trajectories with an 82.6% accuracy. Scaling the pipeline, agents based on Llama 3.1 70B solve 16.7% of tasks for 150k sites. Training on the data generated by our pipeline is competitive with training on human demonstrations. In data-limited settings derived from Mind2Web and WebLINX, we improve Step Accuracy by up to +89.5% and +122.1% respectively for agents trained on mixtures of data from our pipeline, and human data. When training agents with all available human data from these benchmarks, agents fail to generalize to diverse real sites, and adding our data improves their generalization by +149.0% for WebLINX and +156.3% for Mind2Web. Code available at: [data-for-agents.github.io](https://data-for-agents.github.io).

[website](https://data-for-agents.github.io)    |    [paper](https://arxiv.org/abs/2502.06776)    |    [data](https://huggingface.co/datasets/data-for-agents/insta-150k)

---

This notebook provides a demo of the InSTA pipeline for agents.

In [1]:
# Run the following commands to install InSTA and prepare the environment

# !docker pull brandontrabucco/insta-browser-environment
# !docker run -p 7860:7860 -p 3000-3007:3000-3007 -t brandontrabucco/insta-browser-environment &
# !pip install git+https://github.com/data-for-agents/insta

In [2]:
from insta import (
    InstaPipeline,
    create_demo_videos
)

In [3]:
pipeline = InstaPipeline()

In [4]:
N = 0  # select the number of demos

for data_idx, trajectory in enumerate(
    pipeline.iter_pipeline()
):

    if data_idx >= N:
        
        break

Processing: research-portal.uws.ac.uk:   0%|                                   | 1/143811 [13:59<33542:37:36, 839.67s/it]


In [4]:
create_demo_videos(
    task_is_feasible_threshold = 1.0,
    success_threshold = 1.0,
    on_right_track_threshold = 1.0,
)

Processing: rubytalk.org: 100%|████████████████████████████████████████████████████████| 234/234 [01:18<00:00,  3.00it/s]


In [5]:
from ipywidgets import Output, GridspecLayout
from IPython import display

import glob
import json
import os

MAX_VIDEOS = 10

selected_video_files = glob.glob(
    "data/videos/*.mp4"
)[:MAX_VIDEOS]

height = len(selected_video_files)

video_grid = GridspecLayout(
    1, height
)

for panel_idx, video_path in enumerate(
    selected_video_files
):
    
    video_output = Output()
    
    with video_output:
        
        display.display(display.Video(
            video_path,
            embed = True,
            width = 1000
        ))
        
    video_grid[0, panel_idx] = video_output

video_grid

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [6]:
first_video_path = selected_video_files[0]

# show the final observation, action, and judgment responses:

final_observation = "data/observations/{}".format(
    os.path.basename(first_video_path)
    .replace(".mp4", ".json")
)

with open(final_observation, "r") as file:

    final_observation = json.load(file)[-1]

final_action = "data/actions/{}".format(
    os.path.basename(first_video_path)
    .replace(".mp4", ".json")
)

with open(final_action, "r") as file:

    final_action = json.load(file)[-1]

judgment = "data/judgments/{}".format(
    os.path.basename(first_video_path)
    .replace(".mp4", ".json")
)

with open(judgment, "r") as file:

    judgment = json.load(file)

print(
    "Final Observation for {}:\n\n{}\n\n".format(
        first_video_path,
        final_observation["processed_text"]
    )
)

print(
    "Final Action for {}:\n\n{}\n\n".format(
        first_video_path,
        final_action["response"]
    )
)

print(
    "Judgment for {}:\n\n{}\n\n".format(
        first_video_path,
        judgment["response"]
    )
)

Final Observation for data/videos/apple.es.mp4:

iPhone 16 y iPhone 16 Plus - Especificaciones técnicas - Apple (ES) [id: 823] iPhone 16 link 
* [id: 832] Descripción link
* [id: 834] Pasarse de Android al iPhone link
* Especificaciones
 [id: 846] Comprar un iPhone 16 link | Negro, blanco, rosa, verde azulado y azul ultramar | Diseño de aluminio | Parte delantera con Ceramic Shield de última generación | Parte trasera de vidrio tintado en masa (negro, rosa, verde azulado y azul ultramar) | Negro, blanco, rosa, verde azulado y azul ultramar | Diseño de aluminio | Parte delantera con Ceramic Shield de última generación | Parte trasera de vidrio tintado en masa (negro, rosa, verde azulado y azul ultramar) | | Capacidad [id: 886] Nota a pie de página 1 link | 128 GB | 256 GB | 512 GB | 128 GB | 256 GB | 512 GB | | Dimensiones y peso [id: 909] Nota a pie de página 2 link | Ancho: | 7,16 cm | Alto: | 14,76 cm | Grosor: | 0,78 cm | Peso: | 170 g | Ancho: | 7,78 cm | Alto: | 16,09 cm | Grosor: